<a href="https://colab.research.google.com/github/shakeeluetp1041/Machine-Learning/blob/main/FullyConnectedNeuralNetworkForFashionMNISTClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code for Classification of FashionMNIST datsets using the fully connected neural network

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import libraries
import torch
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
device='cuda' if torch.cuda.is_available() else 'cpu'
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from pathlib import Path
import matplotlib.pyplot as plt
import random
from torch import nn
from time import time
from tqdm.auto import tqdm

In [ ]:
# Dowload dataset
train_dataset = datasets.FashionMNIST(root='data', train = True, transform= transforms.ToTensor(), target_transform= None, download=True)
test_dataset = datasets.FashionMNIST(root='data', train = False, transform= transforms.ToTensor(), target_transform= None, download=True)


100%|██████████| 26.4M/26.4M [00:02<00:00, 11.2MB/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 200kB/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.75MB/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 15.5MB/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
BATCH_SIZE = 32  # Set the desired batch size
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
Sample_Images,Sample_Labels= next(iter(train_loader))


In [ ]:
# Function to calculate the accuracay
def accuracy_fn(y_true,y_predicted):
  count=torch.eq(y_true,y_predicted).sum().item()
  acc=(count/len(y_true))*100
  return acc
# Neural network archetecture
class FashionMNISTClassification(nn.Module):
  def __init__(self,INPUT_UNITS,HIDDEN_UNITS,OUTPUT_UNITS):
    super().__init__()
    self.layer=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=INPUT_UNITS,out_features=HIDDEN_UNITS),
        nn.Linear(in_features=HIDDEN_UNITS,out_features=OUTPUT_UNITS)
    )
  def forward (self,x):
    return self.layer(x)

INPUT_UNITS=len(Sample_Images[0][0,:,0])*len(Sample_Images[0][0,0,:])
HIDDEN_UNITS=128
OUTPUT_UNITS=len(train_dataset.classes)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
# Model Instentiation
model=FashionMNISTClassification(INPUT_UNITS,HIDDEN_UNITS,OUTPUT_UNITS)

In [ ]:
#Defive the loss function for multiclass classification
loss_fn=nn.CrossEntropyLoss()
# Define Optimizer (stochastic gradinet descent SGD)
optimizer=torch.optim.SGD(params=model.parameters(),lr=0.1)

start_time=time() # Start the timer
epochs=20
for epoch in tqdm(range(epochs)): # tqdm for the progress bar
  model.train() # Training loop start here
  running_train_loss=0
  running_train_acc=0
  for Images_train,Labels_train in train_loader: # Batch loop start here
    y_train_logits=model(Images_train)
    y_train_probs=torch.softmax(y_train_logits,dim=1)
    y_train_pred_labels=torch.argmax(y_train_probs,dim=1)
    train_batch_loss=loss_fn(y_train_logits,Labels_train)
    running_train_loss+=train_batch_loss
    train_batch_acc=accuracy_fn(y_true=Labels_train,y_predicted=y_train_pred_labels)
    running_train_acc+=train_batch_acc
    optimizer.zero_grad()
    train_batch_loss.backward()
    optimizer.step()
  train_loss=running_train_loss/len(train_loader)
  train_acc=running_train_acc/len(train_loader)
  model.eval() # Evaluation mode start here, evaluate after each epoch
  with torch.inference_mode(): # Inference mode start
    running_test_loss=0
    running_test_acc=0
    for Images_test,Labels_test in test_loader:
      y_test_logits=model(Images_test)
      y_test_probs=torch.softmax(y_test_logits,dim=1)
      y_test_pred_labels=torch.argmax(y_test_probs,dim=1)
      test_batch_loss=loss_fn(y_test_logits,Labels_test)
      running_test_loss+=test_batch_loss
      test_batch_acc=accuracy_fn(y_true=Labels_test,y_predicted=y_test_pred_labels)
      running_test_acc+=test_batch_acc
    test_loss=running_test_loss/len(test_loader)
    test_acc=running_test_acc/len(test_loader)

  if epoch%2==0:
    print(f"EPOCHS:{epoch} | Train Loss: {train_loss} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss} | Test Acc: {test_acc:.4f}")
end_time=time()
print(f"Total Time:{end_time-start_time}")

  0%|          | 0/20 [00:00<?, ?it/s]

EPOCHS:0 | Train Loss: 0.5765990018844604 | Train Acc: 79.5600 | Test Loss: 0.5530450344085693 | Test Acc: 80.2716
EPOCHS:2 | Train Loss: 0.45339640974998474 | Train Acc: 84.1100 | Test Loss: 0.4766421616077423 | Test Acc: 83.3866
EPOCHS:4 | Train Loss: 0.43413808941841125 | Train Acc: 84.8633 | Test Loss: 0.5113363265991211 | Test Acc: 80.9105
